In [0]:
from ftplib import FTP
import os
import time

# === CONFIGURACIÓN ===
ftp_host = "ftp.ncbi.nlm.nih.gov"
ftp_dir = "/pubmed/baseline/"
dbfs_target_dir = "/dbfs/FileStore/pubmed_baseline"

batch_size = 100           # Archivos por lote
sleep_seconds = 10         # Pausa entre lotes (segundos)
max_batches = None         # None = todos los lotes posibles, o pon un número para limitar

# Crear carpeta destino en DBFS si no existe
os.makedirs(dbfs_target_dir, exist_ok=True)

# === OBTENER LISTA DE ARCHIVOS DESDE FTP ===
ftp = FTP(ftp_host)
ftp.login()
ftp.cwd(ftp_dir)
all_files = [f for f in ftp.nlst() if f.endswith(".xml.gz")]
ftp.quit()

selected_files = sorted([f for f in all_files if f.startswith("pubmed25n")])

total_files = len(selected_files)
total_batches = (total_files + batch_size - 1) // batch_size

if max_batches:
    total_batches = min(total_batches, max_batches)

print(f"📂 Total archivos: {total_files} | Total lotes: {total_batches} | Tamaño de lote: {batch_size}")

# === FUNCIÓN DE DESCARGA ===
def download_ftp_file(file_name):
    import os
    from ftplib import FTP

    try:
        target_path = f"/dbfs/FileStore/pubmed_baseline/{file_name}"
        if os.path.exists(target_path):
            return f"✔️ {file_name} ya existe"

        ftp = FTP("ftp.ncbi.nlm.nih.gov")
        ftp.login()
        ftp.cwd("/pubmed/baseline/")
        with open(target_path, "wb") as f:
            ftp.retrbinary(f"RETR " + file_name, f.write)
        ftp.quit()
        return f"✅ {file_name} descargado"
    except Exception as e:
        return f"❌ Error {file_name}: {str(e)}"

# === LOOP POR LOTES ===
for batch_number in range(total_batches):
    start = batch_number * batch_size
    end = min(start + batch_size, total_files)

    batch_files = selected_files[start:end]

    existing_files = {f.name for f in dbutils.fs.ls("dbfs:/FileStore/pubmed_baseline")}
    files_to_download = [f for f in batch_files if f not in existing_files]

    print(f"\n🚀 Lote #{batch_number+1}/{total_batches} -> Archivos {start}–{end-1} ({len(files_to_download)} nuevos)")

    if not files_to_download:
        print("🎉 Todos los archivos de este lote ya están descargados.")
    else:
        rdd = spark.sparkContext.parallelize(files_to_download, numSlices=min(len(files_to_download), batch_size))
        resultados = rdd.map(download_ftp_file).collect()

        for r in resultados:
            print(r)

    if batch_number < total_batches - 1:
        print(f"⏳ Esperando {sleep_seconds} segundos antes del siguiente lote...")
        time.sleep(sleep_seconds)

print("\n✅ Descarga por lotes finalizada.")



📂 Total archivos: 1274 | Total lotes: 13 | Tamaño de lote: 100

🚀 Lote #1/13 -> Archivos 0–99 (0 nuevos)
🎉 Todos los archivos de este lote ya están descargados.
⏳ Esperando 10 segundos antes del siguiente lote...

🚀 Lote #2/13 -> Archivos 100–199 (100 nuevos)
✅ pubmed25n0101.xml.gz descargado
✅ pubmed25n0102.xml.gz descargado
✅ pubmed25n0103.xml.gz descargado
✅ pubmed25n0104.xml.gz descargado
✅ pubmed25n0105.xml.gz descargado
✅ pubmed25n0106.xml.gz descargado
✅ pubmed25n0107.xml.gz descargado
✅ pubmed25n0108.xml.gz descargado
✅ pubmed25n0109.xml.gz descargado
✅ pubmed25n0110.xml.gz descargado
✅ pubmed25n0111.xml.gz descargado
✅ pubmed25n0112.xml.gz descargado
✅ pubmed25n0113.xml.gz descargado
✅ pubmed25n0114.xml.gz descargado
✅ pubmed25n0115.xml.gz descargado
✅ pubmed25n0116.xml.gz descargado
✅ pubmed25n0117.xml.gz descargado
✅ pubmed25n0118.xml.gz descargado
✅ pubmed25n0119.xml.gz descargado
✅ pubmed25n0120.xml.gz descargado
✅ pubmed25n0121.xml.gz descargado
✅ pubmed25n0122.xml.gz des

In [0]:
from pyspark.sql.functions import input_file_name


files = dbutils.fs.ls("dbfs:/FileStore/pubmed_baseline")

num_files = len(files)
total_size = sum(f.size for f in files)

print(f"Número de ficheros: {num_files}")
print(f"Tamaño total: {total_size/1024/1024:.2f} MB")

Número de ficheros: 1274
Tamaño total: 47219.78 MB
